### Imports

In [ ]:

import mosaique as mq
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import numpy as np
import pennylane as qml
import os
import time
import datetime
from tensorflow import keras
from mosaique.models.operation import OperationLayer



In [ ]:
# Set the environment for asynchronous GPU usage
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


### Load the Pre-Processed Dataset

Load the dataset and fit the convolution layers.

In [ ]:

mnist_dataset = keras.datasets.mnist
train_layer = mq.ConvolutionLayer4x4("mnist_train5")
test_layer = mq.ConvolutionLayer4x4("mnist_test5")
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()
train_layer.fit(train_images)
test_layer.fit(test_images)
train_images = train_layer.transform(train_images)
test_images = test_layer.transform(test_images)
train_images = train_layer.post_transform(train_images.transpose((0,2,1)))
test_images = test_layer.post_transform(test_images.transpose((0,2,1)))

### Prepare the Model

The model is defined here and an operation for training is prepared here to be pooled later.

Set epochs to 60 for long average.

In [ ]:
def run(tr_images, te_images, label):
    log_dir = train_layer.name + "/run5/" + label
    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True,
        write_steps_per_second=True,
        update_freq='batch',
        profile_batch=1,
        embeddings_freq=1,
        embeddings_metadata=None
    )
    q_model = keras.models.Sequential([
        keras.layers.Rescaling(scale=-1. / 127.5, offset=1),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    q_model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    q_history = q_model.fit(
        tr_images,
        train_labels,
        validation_data=(te_images, test_labels),
        batch_size=128,
        epochs=60,
        verbose=2,
        callbacks=[tensorboard_callback]
    )

def model(variant, tr_layer, te_layer):
    tr_images = tr_layer.open(variant)
    te_images =  te_layer.open(variant)

    label = ''.join(map(str,variant))

    run(tr_images, te_images, label)

### Prepare Permutations

Generate the permutations of the wire assignments.

In [ ]:
permutations = np.asarray(list(itertools.permutations(range(4))))

### Visual Validation

Select a wire assignment to visually review.

Display the separate channels visual.

In [ ]:
from matplotlib import pyplot as plt

print(train_layer.open([0,1,2,3]).shape)

post = train_layer.open([0,1,2,3])

_min, _max = np.amin(post), np.amax(post)
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Plot all output channels for quantum cnot
for c in range(4):
    axes[c].imshow(post[0,:,:,c],vmin = _min, vmax = _max)

Display the merged channels visual.

In [ ]:
plt.imshow((train_layer.channel_merge(post))[0,:,:],vmin = _min, vmax = _max)

### Train Permutations In Parallel

In [ ]:


for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(model,variant = p, tr_layer = train_layer, te_layer= test_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
# 1 min 8 sec

Include the unfiltered control.

In [ ]:
run(train_images,test_images,"NO-FILTER")

### View and Process Data from TensorBoard

In [ ]:
%load_ext tensorboard

![image.png](image.png)